<a href="https://colab.research.google.com/github/laura314159265/poly/blob/main/SIMULACI%C3%93_A/Errors_Terra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import numpy as np
import matplotlib.pyplot as plt

# Dades en unitats del SI
G= 6.67430e-11
m_sol= 1.98847e30
m_terra= 5.97219e24
m_lluna= 7.349e22

# Normalització
t0= 86400.0            # Temps característic= 1 dia
m0= m_sol              # Massa característica= massa del sol
r0= (G * t0**2 * m0)**(1/3)  # Radi característic a partir de les dues constants anteriors

def normalitza(cos): # Primer passem a m i m/s, després normalitzem
  pos= cos[:3] * 1e3 / r0
  vel= cos[3:] * 1e3 * (t0 / r0)
  return pos, vel

def acceleracions(pos, masses): # pos és matriu N files i 3 columnes
    N= len(masses) # Nombre de planetes
    acc= np.zeros_like(pos) # Crea un array amb la mateixa forma que pos però ple de zeros
    for i in range(N):
        for j in range(N):
            if i != j:  # Si i ≠ j, perquè un cos no s'atrau a ell mateix
                diff= pos[j] - pos[i]
                d= np.linalg.norm(diff) # Calcula la norma del vector diferència de posicions
                acc[i]= acc[i] + masses[j] * diff / d**3
    return acc
def euler(pos, vel, masses, h):
    acc_euler= acceleracions(pos, masses)
    vel_nou= vel + h * acc_euler
    pos_nou= pos + h * vel
    return pos_nou, vel_nou

def rk4(pos, vel, masses, h):
    K1= vel
    L1= acceleracions(pos, masses)
    K2= vel + 0.5*h*L1
    L2= acceleracions(pos + 0.5*h*K1, masses)
    K3= vel + 0.5*h*L2
    L3= acceleracions(pos + 0.5*h*K2, masses)
    K4= vel + h*L3
    L4= acceleracions(pos + h*K3, masses)
    pos_nou= pos + (h/6)*(K1 + 2*K2 + 2*K3 + K4)
    vel_nou= vel + (h/6)*(L1 + 2*L2 + 2*L3 + L4)
    return pos_nou, vel_nou


In [35]:
from google.colab import files
import pandas as pd

def error_abs(NASA, analitica):
  """
  NASA: np.array, la solució extreta de la pàgina web
  analitica: np.array, calcul analític amb un mètode numèric
  """
  error = NASA - analitica
  return error

uploaded = files.upload()
file_path = '/content/planet_pos.csv'
NASA = np.array([10**3,10**3,10**3])*np.array(pd.read_csv(file_path))


Saving planet_pos.csv to planet_pos (10).csv


In [42]:
#Sol i Terra
terra= np.array([-4.133776193197704E+05, -1.528302631740080E+08, 3.242787546419352E+04, 2.932642969192311E+01, -7.378251225841874E-02, -1.172300943162528E-03])
sol= np.array([-6.709778526787314E+05, -8.035450223548296E+05, 2.366174814126350E+04, 1.263745636318025E-02, -3.212119035252178E-03, -2.265926685896345E-04])



masses= np.array([m_sol/m0, m_terra/m0])
pos_sol, vel_sol= normalitza(sol)
pos_terra, vel_terra= normalitza(terra)
posicions= np.array([pos_sol, pos_terra])
velocitats= np.array([vel_sol, vel_terra])

dies= 365
h= 1/24
passos= int(dies/h)
euler_st= np.zeros((passos+1, 2, 3))
rk4_st  = np.zeros((passos+1, 2, 3))
pos_euler, vel_euler= posicions.copy(), velocitats.copy()
pos_rk4, vel_rk4= posicions.copy(), velocitats.copy()
euler_st[0]= pos_euler
rk4_st[0]  = pos_rk4

for k in range(passos):
    pos_euler, vel_euler= euler(pos_euler, vel_euler, masses, h)
    pos_rk4, vel_rk4= rk4(pos_rk4, vel_rk4, masses, h)
    euler_st[k+1]= pos_euler
    rk4_st[k+1]  = pos_rk4

#ERROR EULER

print(len(NASA))
print(len(np.array([r0,r0,r0])*euler_st[:,1,:]))

print(NASA)
print( np.array([r0,r0,r0])*euler_st[:,1,:])
print(error_abs(NASA, np.array([r0,r0,r0])*euler_st[:,1,:]))


8761
8761
[[-4.13377619e+08 -1.52830263e+11  3.24278755e+07]
 [-3.07802354e+08 -1.52830491e+11  3.24236679e+07]
 [-2.02226889e+08 -1.52830645e+11  3.24194861e+07]
 ...
 [-8.78261799e+08 -1.52822357e+11  2.54272307e+07]
 [-7.72715396e+08 -1.52823090e+11  2.54295314e+07]
 [-6.67169178e+08 -1.52823749e+11  2.54318123e+07]]
[[-4.13377619e+08 -1.52830263e+11  3.24278755e+07]
 [-3.07802472e+08 -1.52830529e+11  3.24236552e+07]
 [-2.02227452e+08 -1.52830720e+11  3.24194306e+07]
 ...
 [-7.81649725e+09 -1.54090098e+11  2.56033233e+07]
 [-7.71153518e+09 -1.54095796e+11  2.55994364e+07]
 [-7.60656952e+09 -1.54101420e+11  2.55955452e+07]]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.18335544e+02  3.73536180e+04  1.27397615e+01]
 [ 5.63059189e+02  7.49971472e+04  5.54668682e+01]
 ...
 [ 6.93823545e+09  1.26774048e+09 -1.76092529e+05]
 [ 6.93881979e+09  1.27270510e+09 -1.69904966e+05]
 [ 6.93940034e+09  1.27767114e+09 -1.63732908e+05]]


In [ ]:
#Sol, Terra i Lluna

#EULER
euler_stl

#RK4
rk_stl

In [ ]:
#Sol, Terra, Lluna i Mart

#EULER
euler_stlm

#RK4
rk4_stlm